# Power Analysis for Bioinformatics Studies

## The Core Question: How Many Samples Do I Need?

This is one of the most common questions in study design. The answer depends on **effect size** - but where does that number come from?

This notebook covers:
1. What effect size means in biological contexts
2. Three approaches to estimating effect size
3. Practical power calculations for a gene expression study
4. Sensitivity analysis: what if our estimate is wrong?

## Scenario: Biomarker Discovery Study

You are designing a study to identify gene expression biomarkers that distinguish **responders** from **non-responders** to a cancer immunotherapy drug.

**Research question:** Are there genes differentially expressed between responders and non-responders?

**Study design:** 
- Two groups: Responders vs Non-responders
- Measure gene expression via RNA-seq
- Test each gene with a t-test (simplified; in practice you'd use DESeq2 or similar)

**The problem:** Samples are expensive (~$500/sample for RNA-seq). How many patients do you need in each group?

In [ ]:
library(pwr)
library(effsize)
library(ggplot2)

---
## What is Effect Size?

**Effect size** quantifies the magnitude of the difference between groups, standardized by variability.

**Cohen's d** for comparing two means:

$$d = \frac{\bar{X}_1 - \bar{X}_2}{s_{pooled}}$$

Where:
- $\bar{X}_1 - \bar{X}_2$ = difference in group means
- $s_{pooled}$ = pooled standard deviation

**Interpretation (Cohen's conventions):**
| d | Interpretation |
|---|----------------|
| 0.2 | Small effect |
| 0.5 | Medium effect |
| 0.8 | Large effect |

**But these are just conventions!** In bioinformatics, we need domain-specific estimates.

---
## Approach 1: Estimate from Prior Research

The **best** source for effect size is published literature on similar studies.

### Example: Literature Search

You find a published paper studying gene expression in immunotherapy response:

> "We identified 47 differentially expressed genes between responders (n=23) and non-responders (n=31). 
> The top biomarker gene PDL1 showed mean log2 expression of 8.2 (SD=1.4) in responders 
> vs 6.8 (SD=1.6) in non-responders."

Let's calculate the effect size from these reported values:

In [ ]:
# Values from the published paper
mean_responders <- 8.2
sd_responders <- 1.4
n_responders <- 23

mean_nonresponders <- 6.8
sd_nonresponders <- 1.6
n_nonresponders <- 31

# Calculate pooled standard deviation
pooled_sd <- sqrt(((n_responders - 1) * sd_responders^2 + 
                   (n_nonresponders - 1) * sd_nonresponders^2) / 
                  (n_responders + n_nonresponders - 2))

# Calculate Cohen's d
cohens_d_literature <- (mean_responders - mean_nonresponders) / pooled_sd

cat("Effect size from literature:\n")
cat("  Mean difference:", mean_responders - mean_nonresponders, "log2 units\n")
cat("  Pooled SD:", round(pooled_sd, 2), "\n")
cat("  Cohen's d:", round(cohens_d_literature, 2), "\n")
cat("  Interpretation: This is a", 
    ifelse(abs(cohens_d_literature) >= 0.8, "LARGE",
           ifelse(abs(cohens_d_literature) >= 0.5, "MEDIUM", "SMALL")), 
    "effect\n")

### Power Calculation Using Literature-Based Effect Size

Now we can calculate sample size needed for our study:

In [ ]:
# Sample size for 80% power
power_result_80 <- pwr.t.test(
  d = cohens_d_literature,
  sig.level = 0.05,
  power = 0.80,
  type = "two.sample",
  alternative = "two.sided"
)

# Sample size for 90% power
power_result_90 <- pwr.t.test(
  d = cohens_d_literature,
  sig.level = 0.05,
  power = 0.90,
  type = "two.sample",
  alternative = "two.sided"
)

cat("Sample size needed per group:\n")
cat("  For 80% power:", ceiling(power_result_80$n), "per group\n")
cat("  For 90% power:", ceiling(power_result_90$n), "per group\n")
cat("\nTotal samples needed:\n")
cat("  For 80% power:", 2 * ceiling(power_result_80$n), "total\n")
cat("  For 90% power:", 2 * ceiling(power_result_90$n), "total\n")

### Caution: Publication Bias

Published effect sizes are often **inflated** because:
1. Studies with larger effects are more likely to be published
2. Significant results may be selected from multiple comparisons
3. Small samples can produce extreme estimates by chance

**Rule of thumb:** Reduce published effect sizes by 20-30% to be conservative.

In [ ]:
# Conservative estimate (reduce by 25%)
conservative_d <- cohens_d_literature * 0.75

power_conservative <- pwr.t.test(
  d = conservative_d,
  sig.level = 0.05,
  power = 0.80,
  type = "two.sample"
)

cat("Conservative estimate (d reduced by 25%):\n")
cat("  Adjusted Cohen's d:", round(conservative_d, 2), "\n")
cat("  Sample size needed:", ceiling(power_conservative$n), "per group\n")
cat("  Total:", 2 * ceiling(power_conservative$n), "samples\n")

---
## Approach 2: Estimate from Pilot Data

If you have preliminary data (e.g., from a small pilot study or public dataset), you can estimate effect size directly.

### Example: Using GEO Dataset as Pilot Data

Let's simulate what you might find from a small public dataset with 10 samples per group:

In [ ]:
# Simulate pilot data (in practice, this would be real data from GEO/TCGA)
set.seed(42)

# Simulating gene expression values (log2 CPM)
# True effect is d = 0.8, but small samples will give noisy estimates
pilot_responders <- rnorm(n = 10, mean = 7.5, sd = 1.5)
pilot_nonresponders <- rnorm(n = 10, mean = 6.5, sd = 1.5)

# Calculate effect size from pilot data
pilot_effect <- cohen.d(pilot_responders, pilot_nonresponders)

cat("Pilot data summary:\n")
cat("  Responders: mean =", round(mean(pilot_responders), 2), 
    ", SD =", round(sd(pilot_responders), 2), "\n")
cat("  Non-responders: mean =", round(mean(pilot_nonresponders), 2), 
    ", SD =", round(sd(pilot_nonresponders), 2), "\n")
cat("\nEffect size estimate:\n")
print(pilot_effect)

In [ ]:
# Note the wide confidence interval on the effect size!
# This uncertainty should inform our power analysis

# Use the lower bound of the CI for conservative planning
d_estimate <- abs(pilot_effect$estimate)
d_lower <- pilot_effect$conf.int[1]  # Lower bound of 95% CI

cat("Effect size from pilot:\n")
cat("  Point estimate: d =", round(d_estimate, 2), "\n")
cat("  95% CI:", round(pilot_effect$conf.int[1], 2), "to", 
    round(pilot_effect$conf.int[2], 2), "\n")

# Power calculation using lower CI bound (conservative)
if(d_lower > 0) {
  power_pilot <- pwr.t.test(d = d_lower, sig.level = 0.05, power = 0.80)
  cat("\nUsing lower CI bound (conservative):\n")
  cat("  Sample size needed:", ceiling(power_pilot$n), "per group\n")
} else {
  cat("\nWarning: Lower CI includes 0 - effect may not be real!\n")
  cat("Consider: larger pilot study or literature-based estimates\n")
}

### Visualize Pilot Data

In [ ]:
# Create data frame for plotting
pilot_df <- data.frame(
  Expression = c(pilot_responders, pilot_nonresponders),
  Group = factor(rep(c("Responder", "Non-responder"), each = 10))
)

ggplot(pilot_df, aes(x = Group, y = Expression, fill = Group)) +
  geom_boxplot(alpha = 0.7) +
  geom_jitter(width = 0.2, alpha = 0.5) +
  labs(title = "Pilot Data: Gene Expression by Response Group",
       subtitle = paste("Cohen's d =", round(d_estimate, 2)),
       y = "Log2 Expression",
       x = "") +
  theme_minimal() +
  theme(legend.position = "none")

---
## Approach 3: Define Minimum Clinically Meaningful Difference

Sometimes, instead of asking "what effect do we expect?", we ask:

**"What is the smallest effect that would be clinically/biologically meaningful?"**

This is particularly useful when:
- No prior studies exist
- You want to ensure you can detect effects that matter

### Example: What Fold-Change Matters?

In gene expression studies, we often think in terms of **fold-change**:
- 1.5-fold change is often considered the minimum meaningful difference
- 2-fold change is a common threshold for "differentially expressed"

Let's convert fold-change to Cohen's d:

In [ ]:
# Function to convert fold-change to Cohen's d
# Assuming log2-transformed data
fc_to_cohens_d <- function(fold_change, cv = 0.3) {
  # fold_change: the fold change (e.g., 1.5 for 1.5-fold)
  # cv: coefficient of variation (SD/mean), typically 0.2-0.4 for gene expression
  
  # Log2 fold change = difference in log2 means
  log2_fc <- log2(fold_change)
  
  # For log-transformed data, SD is approximately mean * CV
  # Typical log2 expression ~7-8, so SD ~ 2-2.5 for CV=0.3
  typical_mean <- 7.5  # log2 CPM
  sd_estimate <- typical_mean * cv
  
  # Cohen's d
  d <- log2_fc / sd_estimate
  
  return(list(log2_fc = log2_fc, sd = sd_estimate, d = d))
}

# Calculate for different fold-changes
fc_values <- c(1.5, 2.0, 3.0)

cat("Converting Fold-Change to Cohen's d:\n")
cat("(Assuming CV = 0.3, typical for gene expression)\n\n")

for(fc in fc_values) {
  result <- fc_to_cohens_d(fc)
  power_calc <- pwr.t.test(d = result$d, sig.level = 0.05, power = 0.80)
  
  cat(sprintf("%.1f-fold change:\n", fc))
  cat(sprintf("  Log2 FC = %.2f\n", result$log2_fc))
  cat(sprintf("  Cohen's d = %.2f\n", result$d))
  cat(sprintf("  Samples needed: %d per group (80%% power)\n\n", ceiling(power_calc$n)))
}

### Key Insight

Notice how sample size increases dramatically for smaller effects:
- Detecting a 3-fold change is relatively easy (~10 samples/group)
- Detecting a 1.5-fold change requires many more samples (~50-60/group)

**This is why it's crucial to define what difference matters before calculating power!**

---
## Sensitivity Analysis: What If We're Wrong?

Effect size estimates are uncertain. A **sensitivity analysis** shows how sample size requirements change across a range of possible effect sizes.

In [ ]:
# Range of possible effect sizes
d_range <- seq(0.3, 1.2, by = 0.1)

# Calculate sample size for each
sensitivity_results <- data.frame(
  effect_size = d_range,
  n_80_power = sapply(d_range, function(d) {
    ceiling(pwr.t.test(d = d, sig.level = 0.05, power = 0.80)$n)
  }),
  n_90_power = sapply(d_range, function(d) {
    ceiling(pwr.t.test(d = d, sig.level = 0.05, power = 0.90)$n)
  })
)

print(sensitivity_results)

In [ ]:
# Visualize sensitivity analysis
library(tidyr)

sensitivity_long <- sensitivity_results %>%
  pivot_longer(cols = c(n_80_power, n_90_power),
               names_to = "power_level",
               values_to = "sample_size") %>%
  mutate(power_level = ifelse(power_level == "n_80_power", "80% Power", "90% Power"))

ggplot(sensitivity_long, aes(x = effect_size, y = sample_size, color = power_level)) +
  geom_line(size = 1.2) +
  geom_point(size = 3) +
  geom_vline(xintercept = cohens_d_literature, linetype = "dashed", alpha = 0.5) +
  annotate("text", x = cohens_d_literature + 0.05, y = 150, 
           label = "Literature\nestimate", hjust = 0, size = 3) +
  labs(title = "Sensitivity Analysis: Sample Size vs Effect Size",
       subtitle = "How many samples do we need if our effect size estimate is wrong?",
       x = "Cohen's d (Effect Size)",
       y = "Sample Size (per group)",
       color = "") +
  theme_minimal() +
  scale_y_continuous(limits = c(0, 200)) +
  theme(legend.position = "bottom")

---
## Summary: Choosing an Effect Size

| Approach | When to Use | Pros | Cons |
|----------|-------------|------|------|
| **Prior literature** | Similar studies exist | Most defensible, peer-reviewed | Publication bias inflates estimates |
| **Pilot data** | You have preliminary data | Specific to your context | Small pilots give noisy estimates |
| **Minimum meaningful difference** | Novel research area | Clinically relevant | Requires domain expertise |

### Best Practices

1. **Use multiple approaches** and compare results
2. **Be conservative** - reduce literature estimates by 20-30%
3. **Report your assumptions** in grant applications and papers
4. **Conduct sensitivity analysis** to show robustness
5. **Consider practical constraints** (budget, available patients)

---
## In-Class Exercise: Design Your Own Study

### Scenario
You are planning a study to compare **DNA methylation levels** at a specific CpG site between patients with **Type 2 Diabetes** and **healthy controls**.

### Literature Review Findings
You found these published results:

| Study | T2D Mean (%) | T2D SD | Control Mean (%) | Control SD | n per group |
|-------|--------------|--------|------------------|------------|-------------|
| Smith et al. | 72.3 | 8.5 | 65.1 | 9.2 | 45 |
| Jones et al. | 68.9 | 10.1 | 62.4 | 8.7 | 32 |
| Lee et al. | 70.5 | 7.8 | 67.2 | 8.9 | 28 |

### Your Tasks

1. Calculate Cohen's d for each of the three published studies
2. What is the average effect size across studies?
3. Apply a 25% reduction for publication bias - what's your conservative estimate?
4. Calculate sample size needed for 80% power using your conservative estimate
5. If your budget allows for only 40 total samples (20 per group), what power would you achieve?
6. Create a sensitivity analysis plot for effect sizes from 0.4 to 1.0

In [ ]:
# Study data from literature
studies <- data.frame(
  study = c("Smith", "Jones", "Lee"),
  t2d_mean = c(72.3, 68.9, 70.5),
  t2d_sd = c(8.5, 10.1, 7.8),
  ctrl_mean = c(65.1, 62.4, 67.2),
  ctrl_sd = c(9.2, 8.7, 8.9),
  n = c(45, 32, 28)
)

# Task 1: Calculate Cohen's d for each study
# Your code here


# Task 2: Average effect size
# Your code here


# Task 3: Conservative estimate (25% reduction)
# Your code here


# Task 4: Sample size for 80% power
# Your code here


# Task 5: Power with n=20 per group
# Your code here


# Task 6: Sensitivity analysis plot
# Your code here


### Discussion Questions

1. Why do the three published studies show different effect sizes? What factors might explain this variability?

2. If your calculated power with n=20 is below 80%, what options do you have?
   - Increase sample size?
   - Accept lower power?
   - Change the research question?

3. How would you justify your sample size choice in a grant application?